# A6: Natural Language Inference using Neural Networks

by Adam Ek, Bill Noble, and others.

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Write all your answers and the code in the appropriate boxes below.


In this lab we will work with neural networks for natural language inference. Our task is: given a premise sentence P and hypothesis H, what entailment relationship holds between them? Is H entailed by P, contradicted by P or neutral towards P?

Given a sentence P, if H definitely describe something true given P then it is an **entailment**. If H describe something that's *maybe* true given P, it's **neutral**, and if H describe something that's definitely *false* given P it's a **contradiction**. 

## 1. Data

We will explore natural language inference using neural networks on the SNLI dataset, described in [1]. The dataset can be downloaded [here](https://nlp.stanford.edu/projects/snli/), but unfortunately that dataset is down at the moment. Instead, we will use the version uploaded to [HuggingFace](https://huggingface.co/datasets/stanfordnlp/snli) available through the `datasets` library. See the [documentation](https://huggingface.co/docs/datasets/v2.19.0/loading#hugging-face-hub) for loading a dataset from the HuggingFace hub.

The (simplified) data is organized as follows:

* Column 1: Premise
* Column 2: Hypothesis
* Column 3: Relation

Like in the previous lab, we'll need to build a dataloader. You can adapt your code from the previous lab to the new dataset. **[1 mark]**

In [9]:
#   LOADING OUR DATA - SNLI DATASET &
#   DOING OUR IMPORTS:
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch.nn as nn
from transformers import BertTokenizer

dataset = load_dataset('stanfordnlp/snli')
ex = dataset['train'][0]
print(ex)

{'premise': 'A person on a horse jumps over a broken down airplane.', 'hypothesis': 'A person is training his horse for a competition.', 'label': 1}


In [2]:
# CREATING OUR DATASET FOR THE DATALOADER (will be easier since we are using a built-in dataset from HuggingFace):
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')   
dataset = dataset.filter(lambda example: example["label"] != -1)
dataset.set_format(type='torch', columns=['premise', 'hypothesis', 'label'])
# our datasets:
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']

def collate_function(batch):
    """Merges a list of samples to form a mini-batch.
    This function will be used as an argument for the DataLoader below."""
    batch_premise = [example['premise'] for example in batch]   # a list of 4 premises, since our batch size is 4
    batch_hypothesis = [example['hypothesis'] for example in batch] # a list of 4 hypotheses, since our batch size is 4
    batch_label = [example['label'] for example in batch] # a list of 4 labels (as tensors), since our batch size is 4

    return {'premise': batch_premise, 'hypothesis': batch_hypothesis, 'label': batch_label}


c:\Users\User\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/550152 [00:00<?, ? examples/s]

Notice that the dataset comes as a dictionary-like object with three splits: `'test'`, `'train'`, and `'validation'`. Each item is a dictionary containing a `'premise'`, `'hypothesis'`, and `'label'`.

In [3]:
# DATALOADERS FOR OUR TRAIN, VALIDATION AND TEST DATA:
dataloader_train = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_function)
dataloader_validation = DataLoader(validation_dataset, batch_size=4, shuffle=True, collate_fn=collate_function)
dataloader_test = DataLoader(test_dataset, batch_size=4, shuffle=True, collate_fn=collate_function)

for batch in dataloader_train:
    print(batch)
    break  

{'premise': ['Playing an acoustic guitar, a young soloist performs.', 'A man riding a unicycle down a paved road', 'An older man is playing a violin on the street.', 'A cat is standing in front of a woman and a dog on a brick road.'], 'hypothesis': ['A young man playing the violin.', 'A clown rides a unicycle down the road.', 'A man plays music.', 'A cat is standing in front of a woman wanting a treat, and a dog on a brick road.'], 'label': [tensor(2), tensor(1), tensor(0), tensor(1)]}


## 2. Tokenization

This data does not come pre-tokenized. Instead of training our own tokenizer, we can use the BERT tokenizer like in the preivous assignment. Even though we aren't using BERT the tokenizer works with any model. See the documentation on [using a pretrained tokenizer](https://huggingface.co/docs/tokenizers/en/quicktour#using-a-pretrained-tokenizer). **[1 mark]**

In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_word_to_idx(data_loader):
    """Goes through a dataloader's batches, transforms tokens into indices, creates a tensor out of the 
    premise and hypothesis. Labels are already tensors so they simply get appended into tokenized_batch as they are."""
    tokenized_batch = []  # tokenized batches get appended here
    for batch in data_loader:
        tokenized_premises = tokenizer(batch['premise'],padding='longest',truncation=True,return_tensors="pt")
        tokenized_hypotheses = tokenizer(batch['hypothesis'],padding='longest',truncation=True,return_tensors="pt")
        # Appending tokenized premise, hypothesis, and label to the tokenized batch
        tokenized_batch.append({
            'premises_input_ids': tokenized_premises['input_ids'],
            'premises_attention_mask': tokenized_premises['attention_mask'],
            #'premises_token_type_ids': tokenized_premises['token_type_ids'],
            'hypotheses_input_ids': tokenized_hypotheses['input_ids'],
            'hypotheses_attention_mask': tokenized_hypotheses['attention_mask'],
            #'hypotheses_token_type_ids': tokenized_hypotheses['token_type_ids'],
            'label': batch['label']
            })

    #print(f"This is the [0] from the tokenized batch: {tokenized_batch[0]}")        
    return tokenized_batch

training_data = tokenize_word_to_idx(dataloader_train)
validation_data = tokenize_word_to_idx(dataloader_validation)
testing_data = tokenize_word_to_idx(dataloader_test)

In [19]:
print(training_data[0])

{'premises_input_ids': tensor([[  101,  2048,  2273,  2893,  3201,  2000,  5607,  1012,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [  101,  2048,  3057, 11214, 22681,  2013,  1037,  2492,  1012,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [  101,  1037,  2177,  1997,  3157,  9567,  2006,  1037,  2754,  2024,
          3491,  3173,  7334,  1998,  4147,  2304,  1998,  2417, 12703,  1012,
           102],
        [  101,  1037,  2450,  2003,  3030,  2006,  1996, 11996,  2559,  2012,
          1037,  3696,  2006,  1037,  2311,  2813,  1012,   102,     0,     0,
             0]]), 'premises_attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1,

## 2. Model

In this part, we'll build the model for predicting the relationship between H and P.

We will process each sentence using an LSTM. Then, we will construct some representation of the sentence. When we have a representation for H and P, we will combine them into one vector which we can use to predict the relationship.

We will train a model described in [2], the BiLSTM with max-pooling model. The procedure for the model is roughly:

    1) Encode the Hypothesis and the Premise using one shared bidirectional LSTM (or two different LSTMS)
    2) Perform max over the tokens in the premise and the hypothesis
    3) Combine the encoded premise and encoded hypothesis into one representation
    4) Predict the relationship 

### Creating a representation of a sentence

Let's first consider step 2 where we perform pooling. There is a builtin function in pytorch for this, but we'll implement it from scratch. 

Let's consider the general case, what we want to do for these methods is apply some function $f$ along dimension $i$, and we want to do this for all $i$'s. As an example we consider the matrix S with size ``(N, D)`` where N is the number of words and D the number of dimensions:

$S = \begin{bmatrix}
    s_{11} & s_{12} & s_{13} & \dots  & s_{1d} \\
    s_{21} & s_{22} & s_{23} & \dots  & s_{2d} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    s_{n1} & s_{n2} & s_{n3} & \dots  & s_{nd}
\end{bmatrix}$

What we want to do is apply our function $f$ on each dimension, taking the input $s_{1d}, s_{2d}, ..., s_{nd}$ and generating the output $x_d$. 

You will implement the max pooling method. When performing max-pooling, $max$ will be the function which selects a _maximum_ value from a vector and $x$ is the output, thus for each dimension $d$ in our output $x$ we get:

\begin{equation}
    x_d = max(s_{1d}, s_{2d}, ..., s_{nd})
\end{equation}

This operation will reduce a batch of size ``(batch_size, num_words, dimensions)`` to ``(batch_size, dimensions)`` meaning that we now have created a sentence representation based on the content of the representation at each token position. 

Create a function that takes as input a tensor of size ``(batch_size, num_words, dimensions)`` then performs max pooling and returns the result (the output should be of size: ```(batch_size, dimensions)```). [**4 Marks**]

In [5]:
import torch

def max_pooling(input_tensor):
  output_tensor,_ = torch.max(input_tensor, dim=1)
  return output_tensor

test_unpooled = torch.rand(32, 100, 512)
test_pooled = max_pooling(test_unpooled)
#print(test_pooled.size()) # should be torch.Size([32, 512])
#for batch in training_data[:5]:
#  print(batch['premises_input_ids'])
#  print(max_pooling(batch['premises_input_ids']))

### Combining sentence representations

Next, we need to combine the premise and hypothesis into one representation. We will do this by concatenating four tensors (the final size of our tensor $X$ should be ``(batch_size, 4d)`` where ``d`` is the number of dimensions that you use): 

$$X = [P; H; |P-H|; P \cdot H]$$

Here, what we do is concatenating P, H, P times H, and the absolute value of P minus H, then return the result.

Implement the function. **[4 marks]**

In [7]:
def combine_premise_and_hypothesis(hypothesis, premise):
    output = torch.cat((premise, hypothesis, torch.abs(premise - hypothesis), premise * hypothesis), dim=1)
    return output

test_hypothesis = test_pooled.clone()
test_premise = test_pooled.clone()
test_combined = combine_premise_and_hypothesis(test_hypothesis, test_premise) 
print(test_combined.size()) # should be torch.Size([32, 2048])

torch.Size([32, 2048])


In [ ]:
for batch in training_data:
  #print(batch['premises_input_ids'])
  #print(batch['hypotheses_input_ids'])
  premise = max_pooling(batch['premises_input_ids'])#.unsqueeze(0)
  print(premise.shape)
  hypo = max_pooling(batch['hypotheses_input_ids'])#.unsqueeze(0)
  print(hypo.shape)
  #print('premise',premise)
  #print('hypo',hypo)
  x=combine_premise_and_hypothesis(premise,hypo)
  print('combined',x)
  print('size',x.size())
  break
#With the batch size 4 the combined tensor should be (4,16) why isn't it working but the test pool seems correct.

### Creating the model

Finally, we can build the model according to the procedure given previously by using the functions we defined above. Additionaly, in the model you should use *dropout*. For efficiency purposes, it's acceptable to only train the model with either max or mean pooling. 

Implement the model [**8 marks**]

In [10]:
class SNLIModel(nn.Module):
    def __init__(self, vocab_size, hidden_size, embedding_dim, labels_size):
        super(SNLIModel,self).__init__()
        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_size, num_layers=1, bidirectional=False, batch_first=True)
        self.classifier = nn.Linear(in_features=hidden_size*4, out_features=labels_size)
        
    def forward(self, premise, hypothesis):
        p = self.embeddings(premise)
        h = self.embeddings(hypothesis)
        
        p_out,_= self.rnn(p)
        h_out,_ = self.rnn(h)

        p_pooled = max_pooling(p_out)
        h_pooled = max_pooling(h_out)
        
        ph_representation = combine_premise_and_hypothesis(h_pooled,p_pooled)
        predictions = self.classifier(ph_representation)
        
        return predictions

In [11]:
model = SNLIModel(tokenizer.vocab_size,512,512,3)


## 3. Training

As before, implement the training and testing of the model. SNLI can take a very long time to train, so I suggest you only run it for one or two epochs. **[10 marks]** 

**Tip for efficiency:** *when developing your model, try training and testing the model on one batch (for each epoch) of data to make sure everything works! It's very annoying if you train for N epochs to find out that something went wrong when testing the model, or to find that something goes wrong when moving from epoch 0 to epoch 1.*

In [13]:
import torch.optim as optim
from torch.nn import CrossEntropyLoss
epochs = 2
batch_size = 32


loss_function = CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(),lr=0.001)
model = model

for epoch in range(epochs):
    total_loss = 0
    for batch in training_data:
        optimizer.zero_grad()
        premises = batch['premises_input_ids']
        hypothesis = batch['hypotheses_input_ids']
        labels = torch.tensor(batch['label'])     
        #print(labels)
        outputs = model(premises,hypothesis)
        #print(outputs)
        loss = loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss : {total_loss/len(training_data)}')    
# test model after all epochs are completed
correct = 0
with torch.no_grad():
    for batch in testing_data:
        premises = batch['premises_input_ids']
        hypothesis = batch['hypotheses_input_ids']
        labels = torch.tensor(batch['label']) 
        outputs = model(premises,hypothesis)
        max_prediction = torch.argmax(outputs,dim=1)
        correct+=torch.sum(max_prediction == labels).item()
    print(correct)
print("accuracy:", correct/(len(testing_data)*4))
#print(len(testing_data))
        


KeyboardInterrupt: 

## 4. Testing

Test the model on the testset. For each example in the test set, compute a prediction from the model (`entailment`, `contradiction` or `neutral`). Compute precision, recall, and F1 score for each label. **[10 marks]**

Suggest a _baseline_ that we can compare our model against **[2 marks]**

**Your answer should go here**

Suggest some ways (other than using a baseline) in which we can analyse the models performance **[3 marks]**.

**Your answer should go here**

Suggest some ways to improve the model **[3 marks]**.

**Your answer should go here**

## Readings

[1] Samuel R. Bowman, Gabor Angeli, Christopher Potts, and Christopher D. Manning. 2015. A large annotated corpus for learning natural language inference. In Proceedings of the 2015 Conference on Empirical Methods in Natural Language Processing (EMNLP). 

[2] Conneau, A., Kiela, D., Schwenk, H., Barrault, L., & Bordes, A. (2017). Supervised learning of universal sentence representations from natural language inference data. arXiv preprint arXiv:1705.02364.

## Statement of contribution

Briefly state how many times you have met for discussions, who was present, to what degree each member contributed to the discussion and the final answers you are submitting.

## Marks

This assignment has a total of 23 marks.